In [1]:
import pandas as pd
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [32]:
#load the model scaler ohe
model=load_model('model.h5')

with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    gender=pickle.load(file)

with open('one_hot_encoder_geo.pkl','rb') as file:
    geo_ohe=pickle.load(file)


In [33]:
print(type(geo_ohe))


<class 'sklearn.preprocessing._encoders.OneHotEncoder'>


In [34]:
df=pd.read_csv('Churn_Modelling.csv')
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [35]:
input_data1={
 'CreditScore':[600],
   'Geography':['France'],
    'Gender':['Male'], 
    'Age':[40],
    'Tenure':[3],
    'Balance':[60000],
    'NumOfProducts':[2], 
    'HasCrCard':[1],
    'IsActiveMember':[1], 
    'EstimatedSalary':[50000]
}

In [36]:
input_data={
 'CreditScore':600,
   'Geography':'France',
    'Gender':'Male', 
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2, 
    'HasCrCard':1,
    'IsActiveMember':1, 
    'EstimatedSalary':50000
}

In [37]:
input_data_df=pd.DataFrame(input_data1)
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [38]:
input_data_df.dtypes

CreditScore         int64
Geography          object
Gender             object
Age                 int64
Tenure              int64
Balance             int64
NumOfProducts       int64
HasCrCard           int64
IsActiveMember      int64
EstimatedSalary     int64
dtype: object

In [40]:
input_geo=geo_ohe.transform([[input_data['Geography']]]).toarray()
input_geo

c:\Users\ZAKARIA BOULLAM\OneDrive\Desktop\Project_3(DL)\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[1., 0., 0.]])

In [41]:
geo_encoded_df=pd.DataFrame(input_geo,columns=geo_ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [44]:
input_data_df=pd.DataFrame([input_data])

In [ ]:
input_data_df['Gender']=gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [54]:
# concatenation
input_data_df=pd.concat([input_data_df.drop(columns='Geography',axis=1),geo_encoded_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [56]:
scaler.transform(input_data_df)

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [58]:
prediction=model.predict(input_data_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


In [59]:
prediction_proba=prediction[0][0]

In [60]:
prediction_proba

np.float32(0.19570188)

In [61]:
if prediction_proba>0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
